In [ ]:
"""
Create fishnet
"""

from gasp3.gt.sample import create_fishnet

inShp = r'D:\gisbrary\admin_lmt\CAOP2017\pt_lmt.shp'

outShp = r'C:\gwork\fireloc\datasets\ref\pt_ref_grid.shp'

create_fishnet(inShp, outShp, width=20000, height=20000)

In [ ]:
"""
Count vizinhos cell in fishnet
"""

fishnet = r'C:\gwork\fireloc\datasets\ref\pt_ref_grid.shp'

res_x = 20000
res_y = 20000

from gasp3.fm import tbl_to_obj
from gasp3.g.gop.prox  import df_buffer_extent

fish_df = tbl_to_obj(fishnet)

bf_df = df_buffer_extent(grid_df, epsg, cell_width, mantainOriginalGeom=None)

In [ ]:
"""
Create N Fishnet from raster

Fishnet properties will be extracted from raster, namely cellsize
"""

import os
from osgeo import gdal
from gasp.g.prop.img import get_cell_size, rst_epsg
from gasp.g.smp import fishnet

rst = '/mnt/d/gwork/osm2lulc_imgcls/lmt/rst_lisboa.tif'
max_row = 500
max_col = 500
out_folder = '/mnt/d/gwork/osm2lulc_imgcls/fishnet'

# Open Raster
img = gdal.Open(rst)

# Get EPSG
epsg = rst_epsg(img)

# Get Cellsize
tl_x, cs_x, xr, tl_y, yr, cs_y = img.GetGeoTransform()

# Get N cols and Rows
numimg = img.ReadAsArray()
nrows = numimg.shape[0]
ncols = numimg.shape[1]

# Get raster max_x and min_y
rst_max_x = tl_x + (ncols * cs_x)
rst_min_y = tl_y + (nrows * cs_y)

In [ ]:
# Fishnet N
fnrows = int(round(nrows / max_row, 0))
fnrows = fnrows if fnrows == nrows / max_row else fnrows + 1
fncols = int(round(ncols / max_col, 0))
fncols = fncols if fncols == ncols / max_col else fncols + 1

fi = 1
for i in range(fnrows):
    # TopLeft Y
    tly = tl_y + ((max_row * cs_y) * i)
    # BottomRight Y
    bry = tly + (cs_y * max_row)
    # If fishnet min y is lesser than raster min_y
    # Use raster min_y
    if bry < rst_min_y:
        bry = rst_min_y
    
    for e in range(fncols):
        # TopLeft X 
        tlx = tl_x + ((max_col * cs_x) * e)
        # BottomRight X
        brx = tlx + (cs_x * max_col)
        
        # If fishnet max x is greater than raster max_x
        # Use raster max_x
        if brx > rst_max_x:
            brx = rst_max_x
        fishnet(
            (tlx, tly), (brx, bry),
            os.path.join(out_folder, 'fishnet_{}.shp'.format(str(fi))),
            cs_x, abs(cs_y), epsg=epsg
        )
        
        fi += 1